# Stanley Steering Controller

---

## What Is Stanley Control?

Stanley control is a geometric path tracking algorithm used for steering control in autonomous vehicles. It was developed by Stanford University’s team during the DARPA Grand Challenge.

Stanley Control uses the front axle as the reference point and accounts for:

- Heading error (ψ): the difference between the vehicle’s orientation and the path’s direction.
- Cross-track error (e): the lateral distance from the front axle of the car to the closest point on the path.

The goal is to compute a steering angle δ(t) that aligns the car with the path over time.

---

## Geometric Setup

Given:

- (x, y): rear axle position of the vehicle  
- ψ: Yaw error or heading error = θ - car_yaw
- v: vehicle speed  
- (xf, yf): front axle position = (x + L cos(car_yaw), y + L sin(car_yaw))  
- θ: tangent (heading) of the path at the nearest point  

The control law combines both error terms into the steering command:

$
\delta(t) = k_y \psi(t) + \tan^{-1}\left( \frac{k \cdot e(t)}{v + k_s} \right)
$

Where:

- k is a gain parameter for how strongly the cross-track error is corrected.  
- kₛ is a small softening constant to prevent division by zero when speed is low.  
- δ(t) is clipped between steering angle limits.
- k_y is the weight of yaw error which can be useful for tuning the controller

---

## Your Task

Implement the Stanley steering function using the signature below. You will import this function in your animation simulator notebook.

![Stanley Control Diagram](../assets/stanley_diagram.png)
![Stanley Control Formula](../assets/stanley_formula.png)


In [1]:
import numpy as np

def stanley_steering(x, y, yaw, v, waypoints, k=1.0, ks=1e-2, max_steer=np.radians(30)):
    """
    Stanley steering controller.

    Args:
        x, y     : rear axle position of the car
        yaw      : vehicle heading angle (in radians)
        v        : vehicle speed
        waypoints: Nx2 array of path waypoints
        k        : cross-track gain
        ks       : softening term to prevent div by zero
        max_steer: steering angle limits (in radians)

    Returns:
        steer       : steering angle in radians
        target_idx  : index of the nearest waypoint
    """
    # Step 1: Compute front axle position
    L = 2.5  # assume fixed wheelbase
    fx = x + L * np.cos(yaw)
    fy = y + L * np.sin(yaw)

    # Step 2: Find nearest waypoint
    x_dist = waypoints[:, 0] - fx
    y_dist = waypoints[:, 1] - fy
    dist = np.sqrt(x_dist**2 , y_dist**2)
    target_idx = np.argmin(dist)


    # Step 3: Compute heading of path at that point
    next_waypoint = waypoints[target_idx+1]
    path_dx = next_waypoint[0] - waypoints[target_idx][0] #getting slope of the path using  2points
    path_dy = next_waypoint[1] - waypoints[target_idx][1]
    path_yaw = np.arctan2(path_dy, path_dx)


    # Step 4: Compute heading error
    heading_error = path_yaw - yaw



    # Step 5: Compute signed cross-track error
    m = np.tan(path_yaw)
    c = waypoints[target_idx][1] - m * waypoints[target_idx][0]
    cross_track_error = abs(m * x[0] - x[1] + c) / np.sqrt(m**2 + 1) # perpendiuclar distance from a line formula 


    # Step 6: Compute steering using Stanley law
    steer = heading_error + np.arctan2(k * cross_track_error, v + ks) # write your code here
    steer = np.clip(steer, -max_steer, max_steer)

    return steer, target_idx
